# Deep Markov Model for full localization

In [ ]:
from tqdm import tqdm

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from tensorboardX import SummaryWriter
import numpy as np

In [ ]:
batch_size = 128#128
epochs = 600
seed = 1
torch.manual_seed(seed)

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
#計測モデルとか
def get_ot(st,lmap,max_range):
    dis = torch.sqrt((st[:,0]-lmap[0])**2+(st[:,1]-lmap[1])**2)
    angle = torch.atan2((lmap[1]-st[:,1]),(lmap[0]-st[:,0]))-st[:,2]
    return torch.stack([dis,angle],1)
    
def get_all_ot(st,lmap,max_range):
    measure = get_ot(st,lmap[0],max_range)
    for l in range(1,len(lmap)):
        measure = torch.cat([measure, get_ot(st,lmap[l],max_range)],1)
    return torch.tensor(measure)

In [ ]:
landmark_num = 10
start_pos = [2.0,4.0,0.0]#x0,y0,yaw0

In [ ]:
landmark_dim = 2
x_dim = landmark_num*2
h_dim = 32 #32
hidden_dim = 32 #32
z_dim = 3
u_dim = 2
t_max = 139

In [ ]:
#データの読み込み
transform = transforms.Compose([transforms.ToTensor()])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

#data loader #とりあえず1時系列分を分身させて食わせてる
#[time,s_x,s_y,s_yaw,uv,ur,ot[1],,,,ot[N]]
data = np.loadtxt('vehicle_motion_data.csv', delimiter=',')
data = torch.tensor([data],dtype=torch.float32)
st = data[0,:,1:(1+z_dim)]
ut = data[0,:,(1+z_dim):(1+z_dim+u_dim)]
ot = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
t_max = len(ot)

print(st.size())
st=st.repeat(1000,1,1)
ut=ut.repeat(1000,1,1)
ot=ot.repeat(1000,1,1)
print(st.size())


landmark = np.loadtxt('landmark_data.csv',delimiter=',')

train = torch.utils.data.TensorDataset(ot,ut)
train_loader = torch.utils.data.DataLoader(train, shuffle=False,**kwargs)
test = torch.utils.data.TensorDataset(ot,ut)
test_loader = torch.utils.data.DataLoader(test, shuffle=False,**kwargs)

In [ ]:
from pixyz.models import Model
from pixyz.losses import KullbackLeibler, CrossEntropy, IterativeLoss
from pixyz.distributions import Bernoulli, Normal, Deterministic
from pixyz.utils import print_latex

In [ ]:
class RNN(Deterministic):
    def __init__(self):
        super(RNN, self).__init__(cond_var=["x"], var=["h"])
        self.rnn = nn.GRU(x_dim, h_dim, bidirectional=True)
#         self.h0 = torch.zeros(2, batch_size, self.rnn.hidden_size).to(device)
        self.h0 = nn.Parameter(torch.zeros(2, 1, self.rnn.hidden_size))
        self.hidden_size = self.rnn.hidden_size
        
    def forward(self, x):
        h0 = self.h0.expand(2, x.size(1), self.rnn.hidden_size).contiguous()
        h, _ = self.rnn(x, h0)
        return {"h": h}

In [ ]:
# class Generator(Bernoulli):
#     def __init__(self):
#         super(Generator, self).__init__(cond_var=["z"], var=["x"])
#         self.fc1 = nn.Linear(z_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, x_dim)
    
#     def forward(self, z):
#         print(z.size()) #[128,3]
#         h = F.relu(self.fc1(z))
#         return {"probs": torch.sigmoid(self.fc2(h))}
class Generator(Normal):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"])
    
    def forward(self, z):#計測モデルそのまま
        ot=get_all_ot(z,landmark,[1000,1000])
        return {"loc": ot,"scale":torch.tensor(0.3).to(device)}

In [ ]:
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["h", "z_prev", "u"], var=["z"])
        self.fc1 = nn.Linear(z_dim, h_dim*2)
        self.fc2 = nn.Linear(u_dim, h_dim*2)
        self.fc31 = nn.Linear(h_dim*2, z_dim)
        self.fc32 = nn.Linear(h_dim*2, z_dim)
        
    def forward(self, h, z_prev, u):
        h_z = torch.tanh(self.fc1(z_prev))
        h_u = torch.tanh(self.fc2(u))
        h = (1.0/3.0) * (h + h_z + h_u)
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

In [ ]:
class Prior(Normal):
    def __init__(self):
        super(Prior, self).__init__(cond_var=["z_prev","u"], var=["z"])
        
    def forward(self, z_prev, u):
        # motion model for two-wheel robot x,y,orient,v,steering
        z = torch.zeros(len(z_prev),z_dim).to(device)
        z[:,2] = z_prev[:,2] + u[:,1]
        z[:,0] = z_prev[:,0] + u[:,0] * torch.cos(z_prev[:,2] + u[:,1])
        z[:,1] = z_prev[:,1] + u[:,0] * torch.sin(z_prev[:,2] + u[:,1])

        return {"loc": z, "scale": torch.tensor(0.3).to(device)}

In [ ]:
prior = Prior().to(device)
encoder = Inference().to(device)
decoder = Generator().to(device)
rnn = RNN().to(device)

In [ ]:
print(prior)
print("*"*80)
print(encoder)
print("*"*80)
print(decoder)
print("*"*80)
print(rnn)

In [ ]:
generate_from_prior = prior * decoder
print(generate_from_prior)
print_latex(generate_from_prior)

In [ ]:
step_loss = CrossEntropy(encoder, decoder) + KullbackLeibler(encoder, prior)
_loss = IterativeLoss(step_loss, max_iter=t_max, 
                      series_var=["x", "h", "u"], update_value={"z": "z_prev"})
loss = _loss.expectation(rnn).mean()

In [ ]:
dmm = Model(loss, distributions=[rnn, encoder, decoder, prior], 
            optimizer=optim.RMSprop, optimizer_params={"lr": 5e-4}, clip_grad_value=10)

In [ ]:
print(dmm)
print_latex(dmm)

In [ ]:
def data_loop(epoch, loader, model, device, train_mode=False):
    mean_loss = 0
    for idx,[o,u] in enumerate(tqdm(loader)):#batch_idx, (data, _) in enumerate(tqdm(loader)):
        o = o.to(device)
        u = u.to(device)
        batch_size = o.size()[0]
        x = o.transpose(0, 1) #多分転置してるだけ
        u = u.transpose(0, 1)
        z_prev = torch.tensor(start_pos)#初期姿勢
        z_prev = z_prev.repeat(batch_size, 1).to(device)
        if train_mode:
            mean_loss += model.train({'x': x, 'u':u, 'z_prev': z_prev}).item() * batch_size
        else:
            mean_loss += model.test({'x': x, 'u':u, 'z_prev': z_prev}).item() * batch_size
    mean_loss /= len(loader.dataset)
    if train_mode:
        print('Epoch: {} Train loss: {:.4f}'.format(epoch, mean_loss))
    else:
        print('Test loss: {:.4f}'.format(mean_loss))
    return mean_loss

In [ ]:
def plot_image_from_latent(batch_size):
    x = []
    z_prev = torch.zeros(batch_size, z_dim).to(device)
    u0 = torch.zeros(batch_size, u_dim).to(device)
    for step in range(t_max):
        samples = generate_from_prior.sample({'z_prev': z_prev,'u':u0})
        x_t = decoder.sample_mean({"z": samples["z"]})
        z_prev = samples["z"]
        x.append(x_t[None, :])
    x = torch.cat(x, dim=0).transpose(0, 1)
    return x

In [ ]:
writer = SummaryWriter()

history = {"train_loss":[],"test_loss":[]}

for epoch in range(1, epochs + 1):
    train_loss = data_loop(epoch, train_loader, dmm, device, train_mode=True)
    test_loss = data_loop(epoch, test_loader, dmm, device)

    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('test_loss', test_loss, epoch)

    #loss描画用
    history["train_loss"].append(train_loss)
    history["test_loss"].append(test_loss)
    
    sample = plot_image_from_latent(batch_size)[:, None][1,:]
    writer.add_image('Image_from_latent', sample, epoch)

In [ ]:
import matplotlib.pyplot as plt

#Loss plot
plt.ylabel('$loss$', fontsize=16)
plt.xlabel('$epoch$', fontsize=16)
ay=plt.gca()
plt.title("train_loss")
plt.plot(range(epochs), [i+0.5 for i in history["train_loss"]])
plt.show()
ay=plt.gca()
plt.title("test_loss")
plt.plot(range(epochs), [i+0.4 for i in history["test_loss"]])
plt.show()

In [ ]:

inference_net = rnn*encoder
test_o = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
test_o = torch.tensor(test_o).reshape(1,len(test_o),x_dim).to(device)
test_u = data[0,:,(1+z_dim):(1+z_dim+u_dim)]
test_u = torch.tensor(test_u).reshape(1,len(test_u),u_dim).to(device)
z_prev = torch.tensor(start_pos).to(device)
infered_result = inference_net.sample({"x":test_o,"z_prev":z_prev,"u":test_u})["z"].to("cpu")
infered_result=infered_result.numpy()

plt.plot(infered_result[:,:, 0], infered_result[:,:, 1], "co")
plt.show()